In [ ]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import KerasTools as KT

import random
import io

import keras
import numpy as np
import tensorflow as tf
import PIL
import bqplot.pyplot
import ipywidgets

tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
mapping = KT.datasets.decode._mapping['cifar10']
labels = ['Airplane', 'Bird']

#(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar100.load_data(label_mode='fine')
#mapping = KT.datasets.decode._mapping['cifar100_fine']
#labels = ['Roses', 'Sunflowers']

cifar_map = dict()
random.seed(0)

train_pairs = []
test_pairs = []
for i,l in enumerate(labels):
    idx = mapping.index(l)
    cifar_map[i] = l
    idx_train = train_images[train_labels.flatten() == idx]
    idx_test = test_images[test_labels.flatten() == idx]
    train_pairs.extend([(img,i) for img in idx_train])
    test_pairs.extend([(img,i) for img in idx_test])

random.shuffle(train_pairs)
random.shuffle(test_pairs)

train_images, train_labels = zip(*train_pairs)
test_images, test_labels = zip(*test_pairs)
train_images = np.asarray(train_images)
train_labels = np.asarray(train_labels)
test_images = np.asarray(test_images)
test_labels = np.asarray(test_labels)
print(cifar_map)
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

In [ ]:
r, c, s = 6, 8, 72
pics = train_images[np.random.randint(train_images.shape[0], size=r*c)]
canvas = PIL.Image.new('RGB', (c*s+2, r*s+2), color='white')
for i,d in enumerate(pics):
    dimg = PIL.Image.fromarray(d).resize((s-8, s-8))
    canvas.paste(dimg, box=(s*int(i/r), s*(i%r)))

buf = io.BytesIO()
canvas.save(buf, 'gif')
img = ipywidgets.Image(value=buf.getvalue())
display(img)

In [ ]:
x_train = train_images.astype('float32') / 255
x_test = test_images.astype('float32') / 255

y_train = keras.utils.to_categorical(train_labels)
y_test = keras.utils.to_categorical(test_labels)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


In [ ]:
class plot_history(keras.callbacks.Callback):
    def __init__(self, loss_plt, acc_plt, mode='val', history = None):
        self.loss_plt = loss_plt
        self.acc_plt = acc_plt
        self.mode = mode
        if history is None:
            history = {'loss':[], 'val_loss':[], 'acc': [], 'val_acc':[]}
        self.history = history
    def on_epoch_end(self, epoch, logs={}):
        if self.mode == 'val':
            self.history['loss'].append(logs.get('loss'))
            self.history['val_loss'].append(logs.get('val_loss'))
            self.history['acc'].append(logs.get('acc'))
            self.history['val_acc'].append(logs.get('val_acc'))
        
        imbuf = BytesIO()

        plt.plot(range(1, len(self.history['loss'])+1), self.history['loss'], linewidth=2)
        plt.plot(range(1, len(self.history['val_loss'])+1), self.history['val_loss'], linewidth=2)
        if self.mode == 'final':
            plt.plot([epoch+1], [logs.get('val_loss')], 'rx', ms=5.0)
            plt.axvline(epoch+1, ls=':')
        plt.ylabel('Loss')
        plt.xlabel('Epochs')
        plt.savefig(imbuf)
        imbuf.seek(0)
        self.loss_plt.value = imbuf.getvalue()
        plt.clf()
        plt.plot(range(1, len(self.history['acc'])+1), self.history['acc'], linewidth=2)
        plt.plot(range(1, len(self.history['val_acc'])+1), self.history['val_acc'], linewidth=2)
        if self.mode == 'final':
            plt.plot([epoch+1], [logs.get('val_acc')], 'rx', ms=5.0)
            plt.axvline(epoch+1, ls=':')
        plt.ylabel('Accuracy')
        plt.xlabel('Steps')
        plt.savefig(imbuf)
        imbuf.seek(0)
        self.acc_plt.value = imbuf.getvalue()
        plt.clf()
        
        vt = "Validation" if self.mode == 'val' else "Test"
        print("Epoch {: 4d} | Loss: {:5.4f} Accuracy: {:5.4f} | {} Loss: {:5.4f} {} Accuracy {:5.4f}".format(
            epoch+1,logs.get('loss'),logs.get('acc'),vt,logs.get('val_loss'),vt,logs.get('val_acc')))

In [ ]:
def build_network():
    network = keras.models.Sequential(name='CIFAR_CNN')
    network.add(keras.layers.Conv2D(4, 3, padding='same', activation='relu', input_shape=(32,32,3), name='conv1a'))
    network.add(keras.layers.MaxPooling2D(2, name='pool1'))
    network.add(keras.layers.Conv2D(8, 3, padding='same', activation='relu', name='conv2a'))
    network.add(keras.layers.MaxPooling2D(2, name='pool2'))
    network.add(keras.layers.Conv2D(16, 3, padding='same', activation='relu', name='conv3a'))
    network.add(keras.layers.MaxPooling2D(2, name='pool3'))
    network.add(keras.layers.Conv2D(32, 3, padding='same', activation='relu', name='conv4a'))
    network.add(keras.layers.GlobalMaxPooling2D(name='globalpool'))
    network.add(keras.layers.Dropout(0.25, name='dropout'))
    network.add(keras.layers.Dense(64, activation='relu', name='dense1'))
    network.add(keras.layers.Dense(len(cifar_map), activation='softmax', name='output'))

    network.compile(optimizer=keras.optimizers.adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return network

In [ ]:
def build_network():
    inp = keras.layers.Input(shape=(32,32,3))
    x = keras.layers.Conv2D(8, 3, strides=1, activation='relu', name='conv1')(inp)
    x = keras.layers.MaxPooling2D(2, name='pool1')(x)
    y = keras.layers.Conv2D(8, 3, strides=1, padding='same', activation='relu', name='res1a')(x)
    y = keras.layers.Conv2D(8, 3, strides=1, padding='same', activation='relu', name='res1b')(y)
    x = keras.layers.add([x, y], name='add1')
    x = keras.layers.Activation('relu', name='activation1')(x)
    
    x = keras.layers.Conv2D(16, 3, strides=1, activation='relu', name='conv2')(x)
    x = keras.layers.MaxPooling2D(2, name='pool2')(x)
    y = keras.layers.Conv2D(16, 3, strides=1, padding='same', activation='relu', name='res2a')(x)
    y = keras.layers.Conv2D(16, 3, strides=1, padding='same', activation='relu', name='res2b')(y)
    x = keras.layers.add([x, y], name='add2')
    x = keras.layers.Activation('relu', name='activation2')(x)

    x = keras.layers.Conv2D(32, 3, strides=1, activation='relu', name='conv3')(x)
    x = keras.layers.MaxPooling2D(2, name='pool3')(x)
    y = keras.layers.Conv2D(32, 3, strides=1, padding='same', activation='relu', name='res3a')(x)
    y = keras.layers.Conv2D(32, 3, strides=1, padding='same', activation='relu', name='res3b')(y)
    x = keras.layers.add([x, y], name='add3')
    x = keras.layers.Activation('relu', name='activation3')(x)

    x = keras.layers.GlobalMaxPooling2D(name='globalpool')(x)
    x = keras.layers.Dropout(0.25, name='dropout')(x)
    out = keras.layers.Dense(4, activation='softmax', name='output')(x)
    network = keras.models.Model(inputs=[inp], outputs=[out])
    
    network.compile(optimizer=keras.optimizers.adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return network

network = build_network()
network.summary()
    

In [ ]:
network = build_network()
network.summary()

loss_plt = ipywidgets.Image()
acc_plt = ipywidgets.Image()
display(ipywidgets.HBox([loss_plt, acc_plt]))

datagen = keras.preprocessing.image.ImageDataGenerator(
    #zca_whitening=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True, 
    fill_mode='nearest',
    validation_split=0.25)
datagen.fit(train_images)

train_generator = datagen.flow(train_images, train_labels, batch_size=32, subset='training')
val_generator = datagen.flow(train_images, train_labels, batch_size=32, subset='validation')
epochs = 100
history = network.fit_generator(train_generator, validation_data=val_generator, use_multiprocessing=True, workers=4,
                                epochs=epochs, verbose=0, callbacks=[plot_history(loss_plt, acc_plt)])

In [ ]:
network = build_network()

loss_plt = ipywidgets.Image()
acc_plt = ipywidgets.Image()
display(ipywidgets.HBox([loss_plt, acc_plt]))

datagen = keras.preprocessing.image.ImageDataGenerator(
    #zca_whitening=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True, 
    fill_mode='nearest',
    validation_split=0.25)
datagen.fit(train_images)

train_generator = datagen.flow(train_images, train_labels, batch_size=32)
final_epochs = 90
network.fit_generator(train_generator, epochs=final_epochs, verbose=0, validation_data=((test_images, test_labels)),
                      use_multiprocessing=True, workers=4, 
                      callbacks=[plot_history(loss_plt, acc_plt, mode='final', history=history.history)])

In [ ]:
entry = 6

prediction = network.predict(test_images[entry:entry+1])
ground_truth = np.argmax(test_labels[entry])
prediction_label = np.argmax(prediction[0])
img = test_images[entry]
plt.figure(figsize=(2,2))
plt.imshow(img)
plt.show()

print("Ground truth: {} / {}".format(ground_truth, cifar_map[ground_truth]))
print("Prediction:   {} / {}".format(prediction_label, cifar_map[prediction_label]))
print('-' * 26)
print("{0:<10} | {1:>6} ".format("Label", "Score"))
print('-' * 26)
for i,r in enumerate(prediction[0]):
    print("{:<10} | {:> 8.2%}".format(cifar_map[i], r))
print('-' * 26)

